In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

data = np.loadtxt('ms.data')
labels = np.loadtxt('ms.labels')
df = pd.DataFrame({
    'Feature1': data[:, 0],
    'Feature2': data[:, 1],
    'Label': labels
})

In [2]:
def get_distribution_stats(data_window):

    return {
        'mean': np.mean(data_window, axis=0),
        'std': np.std(data_window, axis=0),
        'median': np.median(data_window, axis=0),
        'skewness': [np.mean((data_window[:, i] - np.mean(data_window[:, i]))**3 / 
                            np.std(data_window[:, i])**3) for i in range(2)]
    }

window_size = 100  # Parameters
step_size = 50 
threshold = 0.25 

windows = []
drift_scores = []

for i in range(0, len(df)-window_size+1, step_size):
    current_window = df[['Feature1', 'Feature2']].iloc[i:i+window_size].values
    stats = get_distribution_stats(current_window)
    windows.append(stats)

    if i == 0:
        base_stats = stats
        drift_score = 0
    else:
        drift_score = np.sum(np.abs(np.array(stats['mean']) - np.array(base_stats['mean'])))
        drift_score += np.sum(np.abs(np.array(stats['std']) - np.array(base_stats['std']))) * 0.5
        
    drift_scores.append(drift_score)

In [3]:
plt.figure(figsize=(12, 8))

# original data points
plt.subplot(211)
scatter = plt.scatter(df['Feature1'], df['Feature2'], c=df['Label'], cmap='viridis')
plt.colorbar(scatter)
plt.title('Data Points Colored by Label')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')

# drift scores over time
plt.subplot(212)
plt.plot(range(len(drift_scores)), drift_scores, 'b-', linewidth=2)
plt.axhline(y=threshold, color='r', linestyle='--', label=f'Threshold ({threshold})')
plt.fill_between(range(len(drift_scores)), drift_scores, alpha=0.2)
plt.title('Concept Drift Detection Over Time')
plt.xlabel('Window Index')
plt.ylabel('Drift Score')
plt.legend()

plt.tight_layout()
plt.show()